In [1]:
import numpy as np
import cs_vqe as c
import ast
import os

from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
import scipy as sp

In [2]:
from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
from openfermion.ops import QubitOperator

In [3]:
# with open("hamiltonians.txt", 'r') as input_file:
#     hamiltonians = ast.literal_eval(input_file.read())
    
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')


with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

In [4]:
for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [5]:
mol_key = 'H2_6-31G_singlet'  
# mol_key ='H2-O1_STO-3G_singlet'


# currently index 2 is contextual part
# ''''''''''''''''3 is NON contextual part

# join together for full Hamiltonian:
ham = hamiltonians[mol_key][2]
ham.update(hamiltonians[mol_key][3]) # full H 
ham

{'IIIII': 1.5293465322243227,
 'IIIIZ': -0.272118136451076,
 'IIIYY': -0.054942371399896106,
 'IIIZI': -0.272118136451076,
 'IIIZZ': 0.27932036412726724,
 'IIXZZ': 0.02839187635882136,
 'IIZII': -0.4055198682337698,
 'IIZIZ': -0.018268981063105066,
 'IIZXZ': -0.03738940915377441,
 'IIZZI': -0.029919945771762033,
 'IIZZX': 0.0630080420394397,
 'IXIYY': 0.01978010056877635,
 'IXIZZ': -0.00039289419593739876,
 'IXXYY': -0.019724560280490088,
 'IXXZZ': 0.009055897238770372,
 'IXYYI': -0.005056211084212929,
 'IXYYX': 0.019724560280490088,
 'IXZZI': -0.00039289419593739876,
 'IXZZX': 0.005140813195008357,
 'IYIIY': 0.005140813195008357,
 'IYIXY': -0.01978010056877635,
 'IYXIY': 0.005056211084212929,
 'IYXXY': 0.019724560280490088,
 'IYYII': -0.009055897238770372,
 'IYYIX': -0.005056211084212929,
 'IYYXI': 0.005056211084212929,
 'IYYXX': -0.019724560280490088,
 'IZIII': -0.4055198682337698,
 'IZIIZ': -0.029919945771762033,
 'IZIXZ': -0.0630080420394397,
 'IZIZI': -0.018268981063105066,
 'IZIZ

In [6]:
print(f"n_qubits:  {hamiltonians[mol_key][1]}")

n_qubits:  5


# Get non-contextual H

In [7]:
nonH_guesses = c.greedy_dfs(ham, 10, criterion='weight')

nonH = max(nonH_guesses, key=lambda x:len(x)) # largest nonCon part found by dfs alg

Split into:

$$H = H_{c} + H_{nc}$$

In [8]:
nonCon_H = {}
Con_H = {}

for P in ham:
    if P in nonH:
        nonCon_H[P]=ham[P]
    else:
        Con_H[P]=ham[P]
        

## Testing contextuality

In [9]:
print('Is NONcontextual correct:', not c.contextualQ_ham(nonCon_H))
print('Is contextual correct:',c.contextualQ_ham(Con_H))

Is NONcontextual correct: True
Is contextual correct: True


# Classical part of problem!

Take $H_{nc}$ and split into:
- $Z$ = operators that completely comute with all operators in $S$
- $T$ = remaining operators in $S$
    - where $S = Z \cup T$  and $S$ is set of Pauli operators in $H_{nc}$
    
    
- We then split the set $T$ into cliques $C_{1}, C_{2}, ... , C_{|T|}$
    - all ops in a clique commute
    - ops between cliques anti-commute!

In [10]:
bool_flag, Z_list, T_list = c.contextualQ(list(nonCon_H.keys()), verbose=True)

In [11]:
Z_list

['IIIII',
 'IIIIZ',
 'IIZII',
 'IIZIZ',
 'IZIII',
 'IZIIZ',
 'IZZII',
 'ZIIII',
 'ZIIIZ',
 'ZIZII',
 'ZZIII',
 'ZZIIZ',
 'ZZZII',
 'ZZZIZ']

In [12]:
T_list

['IIIZI',
 'IIIZZ',
 'IIZXZ',
 'IIZZI',
 'IZIXZ',
 'IZIZI',
 'IZZXI',
 'IZZXZ',
 'ZIIXZ',
 'ZIIZI',
 'ZIZXI',
 'ZIZXZ',
 'ZZIZI',
 'ZZIZZ',
 'ZZZXZ',
 'ZZZZI']

## Get quasi model

First we define

- $C_{i1}$ = first Pauli in each $C_{i}$ set
- $A_{ij} = C_{ij}C_{1i}$


- $G^{prime} = \{1 P_{i} \;| \; i=1,2,...,|Z| \}$
    - aka all the completely commuting terms with coefficients set to +1!

- We define G to be an independent set of $G^{prime}$
    - where $G \subseteq G^{prime}$


In [13]:
G_list, Ci1_list, all_mappings = c.quasi_model(nonCon_H)

In [14]:
print('non-independent Z list:', Z_list)
print('G (independent) Z list:', G_list)

non-independent Z list: ['IIIII', 'IIIIZ', 'IIZII', 'IIZIZ', 'IZIII', 'IZIIZ', 'IZZII', 'ZIIII', 'ZIIIZ', 'ZIZII', 'ZZIII', 'ZZIIZ', 'ZZZII', 'ZZZIZ']
G (independent) Z list: ['ZIIII', 'IZIII', 'IIZII', 'IIIIZ']


In [15]:
print('all Ci1 terms:', Ci1_list)

all Ci1 terms: ['IIIZI', 'IIZXZ']


$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

In [16]:
# Assemble all the mappings from terms in the Hamiltonian to their products in R:
all_mappings

{'IIIII': [[], [], 1],
 'IIIIZ': [['IIIIZ'], [], 1],
 'IIIZI': [[], ['IIIZI'], 1],
 'IIIZZ': [['IIIIZ'], ['IIIZI'], 1],
 'IIZII': [['IIZII'], [], 1],
 'IIZIZ': [['IIZII', 'IIIIZ'], [], 1],
 'IIZXZ': [[], ['IIZXZ'], 1],
 'IIZZI': [['IIZII'], ['IIIZI'], 1],
 'IZIII': [['IZIII'], [], 1],
 'IZIIZ': [['IZIII', 'IIIIZ'], [], 1],
 'IZIXZ': [['IZIII', 'IIZII'], ['IIZXZ'], 1],
 'IZIZI': [['IZIII'], ['IIIZI'], 1],
 'IZZII': [['IZIII', 'IIZII'], [], 1],
 'IZZXI': [['IZIII', 'IIIIZ'], ['IIZXZ'], 1],
 'IZZXZ': [['IZIII'], ['IIZXZ'], 1],
 'ZIIII': [['ZIIII'], [], 1],
 'ZIIIZ': [['ZIIII', 'IIIIZ'], [], 1],
 'ZIIXZ': [['ZIIII', 'IIZII'], ['IIZXZ'], 1],
 'ZIIZI': [['ZIIII'], ['IIIZI'], 1],
 'ZIZII': [['ZIIII', 'IIZII'], [], 1],
 'ZIZXI': [['ZIIII', 'IIIIZ'], ['IIZXZ'], 1],
 'ZIZXZ': [['ZIIII'], ['IIZXZ'], 1],
 'ZZIII': [['ZIIII', 'IZIII'], [], 1],
 'ZZIIZ': [['ZIIII', 'IZIII', 'IIIIZ'], [], 1],
 'ZZIZI': [['ZIIII', 'IZIII'], ['IIIZI'], 1],
 'ZZIZZ': [['ZIIII', 'IZIII', 'IIIIZ'], ['IIIZI'], 1],
 'ZZZII'

Overall $R$ is basically reduced non-contextual set
- where everything in original non-contextual set can be found by **inference!**

# Function form

$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

- note q to do with $G$
- note r to do with $C_{i1}$

In [17]:
model = [G_list, Ci1_list, all_mappings]

fn_form = c.energy_function_form(nonCon_H, model)

# returns [
#            denstion of q,
#            dimension of r,
#            [coeff, indices of q's, indices of r's, term in Hamiltonian]
#         ]


In [18]:
fn_form

[4,
 2,
 [[1.5293465322243227, [], [], 'IIIII'],
  [-0.272118136451076, [3], [], 'IIIIZ'],
  [-0.272118136451076, [], [0], 'IIIZI'],
  [0.27932036412726724, [3], [0], 'IIIZZ'],
  [-0.4055198682337698, [2], [], 'IIZII'],
  [-0.018268981063105066, [2, 3], [], 'IIZIZ'],
  [-0.03738940915377441, [], [1], 'IIZXZ'],
  [-0.029919945771762033, [2], [0], 'IIZZI'],
  [-0.4055198682337698, [1], [], 'IZIII'],
  [-0.029919945771762033, [1, 3], [], 'IZIIZ'],
  [-0.0630080420394397, [1, 2], [1], 'IZIXZ'],
  [-0.018268981063105066, [1], [0], 'IZIZI'],
  [0.2826730025779857, [1, 2], [], 'IZZII'],
  [-0.012196319613897683, [1, 3], [1], 'IZZXI'],
  [0.08820113157758526, [1], [1], 'IZZXZ'],
  [-1.0457075312510897, [0], [], 'ZIIII'],
  [0.07172093412363195, [0, 3], [], 'ZIIIZ'],
  [-0.0630080420394397, [0, 2], [1], 'ZIIXZ'],
  [0.046085877231641814, [0], [0], 'ZIIZI'],
  [0.22102802962140775, [0, 2], [], 'ZIZII'],
  [-0.012196319613897683, [0, 3], [1], 'ZIZXI'],
  [0.08820113157758526, [0], [1], 'ZIZXZ'],


In [19]:
Energy_function = c.energy_function(fn_form)

In [20]:
import random

### now for the q terms we only have +1 or -1 assignment!
q_variables = [random.choice([1,-1]) for _ in range(fn_form[0])]


### r variables is anything that makes up unit vector!
r_variables = c.angular(np.arange(0,2*np.pi, fn_form[1]))
r_variables

(1.0, -0.0, -0.0, -0.0, 0.0)

In [21]:
 Energy_function(*q_variables,*r_variables)

-0.2112821266871156

find_gs_nonconfunction optimizes above steps by:
1. brute forcing all choices of ```q_variables```
    - ```itertools.product([1,-1],repeat=fn_form[0])```
2. optimizing over ```r_variables``` (in code ```x```)
    - using SciPy optimizer!

In [22]:
model = [G_list, Ci1_list, all_mappings]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

print(lowest_eigenvalue)
print(ground_state_params)

-1.8517678403392313
[[1, 1, 1, -1], [-1.0, -4.388077644981462e-11]]


In [23]:
## check
Energy_function(*ground_state_params[0],*ground_state_params[1]) == lowest_eigenvalue

True

# Now need to rotate Hamiltonian!

We now have non contextual ground state: $(\vec{q}, \vec{r})$

In [24]:
ground_state_params

[[1, 1, 1, -1], [-1.0, -4.388077644981462e-11]]

We can use this result - ground state of $H_{nc}$ -  as a classical estiamte of our ground state of the full Hamiltonian ($H = H_{c} + H_{nc}$)

However we can also obtain a quantum correction using $H_{c}$

By minimizing theenergy of the remaining terms in the Hamiltonian over the quantum states that are **consistent with the noncon-textual ground state**.

To do this we first rotate each $G_{j}$ and $\mathcal{A} = \sum_{i=1}^{N} r_{i}A_{i}$:

In [25]:
model = [G_list, Ci1_list, all_mappings]

print(G_list) # G_j terms!
print(Ci1_list) # mathcal(A)

['ZIIII', 'IZIII', 'IIZII', 'IIIIZ']
['IIIZI', 'IIZXZ']


to SINGLE QUBIT pauli Z operators!

- to map the operators in $G$ to single qubit Pauli operators, we use $\frac{\pi}{2}$ rotations!

- note $\mathcal{A}$ is an anti-commuting set... therefore we can use $N-1$ rotations as in unitary partitioning's sequence of rotations to do this!
    - $R^{\dagger}\mathcal{A} R = \text{single Pauli op}$

# Rotate full Hamiltonian to basis with diagonal noncontextual generators!

function ```diagonalize_epistemic```:
1. first if else statement:
    - if cliques present:
        - first maps A to single Pauli operator (if cliques present)
        - then rotates to diagonlize G union with single Pauli opator of A (hence GuA name!)
    - else if NO cliques present:
        - gets rotations to diagonlize G
        
     - these rotations make up GuA term in code!
2. NEXT code loops over terms in GuA (denoted as g in code)
    - if g is not a single qubit $Z$:
        - code generates code to rotate operator to make g diagonal (rotations)
        - then constructs map of g to single Z (J rotation)
    - Note R is applied to GuA
    
    
#########
- Note rotations are given in Appendix A of https://arxiv.org/pdf/2011.10027.pdf
    - First code checks if g op in GuA is diagonal
        - if so then needs to apply "K" rotation (involving $Y$ and $I$ operators (see pg 11 top) to make it NOT diagononal
    - now operator will be diagnoal!
    - next generate "J" rotation
        - turns non-diagonal operator into a single qubit $Z$ operator!

In [ ]:
# Get sequence of rotations requried to diagonalize the generators for the noncontextual ground state!

Rotations_list, diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops = c.diagonalize_epistemic(model,
                                                                                                            fn_form,
                                                                                                            ground_state_params)

In [ ]:
# rotations to map A to single Pauli operator!
Rotations_list

In [ ]:
# rotations to diagonlize G
diagonalized_generators_GuA

In [ ]:
eigen_vals_nonC_ground_state_GuA_ops

# NEW LCU method

In [ ]:
N_index=0
check_reduction=True
N_Qubits= hamiltonians[mol_key][1]
R_LCU, Rotations_list, diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops= c.diagonalize_epistemic_LCU(
                                                                                  model,
                                                                                  fn_form,
                                                                                  ground_state_params,
                                                                                  N_Qubits,
                                                                                  N_index,
                                                                                  check_reduction=check_reduction)

In [ ]:
R_LCU

In [ ]:
order = list(range(hamiltonians[mol_key][1]))
N_index=0
check_reduction=True
N_Qubits= hamiltonians[mol_key][1]

reduced_H = c.get_reduced_hamiltonians_LCU(Con_H,
                               model,
                               fn_form,
                               ground_state_params,
                               order, 
                               N_Qubits,
                               N_index, 
                               check_reduction=check_reduction)

In [ ]:
len(reduced_H[-1])

In [ ]:
reduced_H[3]

In [ ]:
H = conv_scr.Get_Openfermion_Hamiltonian(reduced_H[-1])
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

In [ ]:
## old way
# Get sequence of rotations requried to diagonalize the generators for the noncontextual ground state!

Rotations_list, diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops = c.diagonalize_epistemic(model,
                                                                                                            fn_form,
                                                                                                            ground_state_params)
len(diagonalized_generators_GuA)
Rotations_list

In [ ]:
### old way
order = list(range(hamiltonians[mol_key][1]))
red_H = c.get_reduced_hamiltonians(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           order)
len(red_H[0])

In [ ]:
len(red_H[0])
red_H[0]

In [ ]:
H = conv_scr.Get_Openfermion_Hamiltonian(red_H[-1])
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

In [26]:
from scipy.sparse.linalg import expm
from Misc_functions import sparse_allclose

R_LCU_QubitOp = QubitOperator()
for P in R_LCU: R_LCU_QubitOp+=P
R_LCU_mat = qubit_operator_sparse(R_LCU_QubitOp, n_qubits=hamiltonians[mol_key][1])



R_SeqRot_QubitOp= conv_scr.convert_op_str(Rotations_list[0][1], 1)
R_SeqRot_mat = qubit_operator_sparse(R_SeqRot_QubitOp, n_qubits=hamiltonians[mol_key][1])

theta_sk=Rotations_list[0][0]
exp_rot = expm(R_SeqRot_mat * theta_sk)# / 2)

sparse_allclose(exp_rot, R_LCU_mat)

NameError: name 'R_LCU' is not defined

# Restricting the Hamiltonian to a contextualsubspace

(Section B of https://arxiv.org/pdf/2011.10027.pdf)

In the rotated basis the Hamiltonian is restricted to the subspace stabilized by the noncontextual generators $G_{j}'$

In [ ]:
print(diagonalized_generators_GuA) # G_j' terms!

The quantum correction is then obtained by minimizing the expectation value of this resticted Hamiltonian!

(over +1 eigenvectors of the remaining non-contextual generators $\mathcal{A}'$)

In [ ]:
print(Ci1_list) # mathcal(A)

- $\mathcal{H}_{1}$ denotes Hilbert space of $n_{1}$ qubits acted on by by the single qubit $G_{j}'$ terms
- $\mathcal{H}_{2}$ denotes Hilbert space of remaining $n_{2}$

Overall full Hilbert space is: $\mathcal{H}=\mathcal{H}_{1} \otimes \mathcal{H}_{2}$

The **contextual Hamiltonian** in this rotated basis is:

$$H_{c}'=\sum_{P \in \mathcal{S_{c}'}} h_{P}P$$

The set of Pauli terms in $H_{c}'$ is $\mathcal{S_{c}'}$, where terms in $\mathcal{S_{c}'}$ act on both $\mathcal{H}_{1}$ and $\mathcal{H}_{2}$ subspaces in general!

We can write $P$ terms as:

$$P=P_{1}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}}$$

$P$ commutes with an element of $G'$ if and only if $P_{1} \otimes \mathcal{I}^{\mathcal{H}_{2}}$ does

As the generators $G'$ act only on $\mathcal{H}_{1}$

If $P$ anticommutes with any element of $G'$ then its expection value in the noncontextual state is zero

Thus any $P$ must commute with all elements of $G'$ and so $P_{1} \otimes \mathcal{I}^{\mathcal{H}_{2}}$ too

As the elements of $G'$ are single-qubit Pauli $Z$ operators acting in $\mathcal{H}_{1}$:

In [ ]:
print(diagonalized_generators_GuA) # G_j' terms!

$P_{1}$ must be a product of such operators!

**As the exepcation value of $P_{1}$ is some $p_{1}= \pm 1$ DETERMINED BY THE NONCONTEXTUAL GROUND STATE**

In [ ]:
eigen_vals_nonC_ground_state_GuA_ops

Let $|\psi_{(\vec{q}, \vec{r})} \rangle$ be any quantum state consistent with the nonconxtual ground state $(\vec{q}, \vec{r})$... aka gives correct expection values of:

In [ ]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

Then the action of any $P$ which allows our contextual correction has the form:

$$P |\psi_{(\vec{q}, \vec{r})} \rangle = \big( P_{1}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}} \big) |\psi_{(\vec{q}, \vec{r})} \rangle$$

$$ = p_{1}\big( \mathcal{I}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}} \big) |\psi_{(\vec{q}, \vec{r})} \rangle$$

- repeating above, but $p_{1}$ is the expectation value of $P_{1}$ determiend by the noncontextual ground state!

Thus we can denote $H_{c}' |_{(\vec{q}, \vec{r})}$ as the restriction of $H_{c}'$ on its action on the noncontextual ground state $(\vec{q}, \vec{r})$:

$$H_{c}' |_{(\vec{q}, \vec{r})} =\sum_{\substack{P \in \mathcal{S_{c}'} \\ \text{s.t.} [P, G_{i}']=0 \\ \forall G'_{i} \in G'}} p_{1}h_{P}\big( \mathcal{I}^{\mathcal{H}_{1}} \otimes P_{2}^{\mathcal{H}_{2}} \big) $$

$$=\mathcal{I}_{\mathcal{H}_{1}} \otimes H_{c}'|_{\mathcal{H}_{2}} $$


where we can write:
$$H_{c}'|_{\mathcal{H}_{2}} = \sum_{\substack{P \in \mathcal{S_{c}'} \\ \text{s.t.} [P, G_{i}']=0 \\ \forall G'_{i} \in G'}} p_{1}h_{P}P_{2}^{\mathcal{H}_{2}}$$



Cleary this Hamiltonian on $n_{2}$ qubits is given by:

$$n_{2} = n - |G|$$

- $|G|=$ number of noncontextual generators $G_{j}$

In [ ]:
from copy import deepcopy
import pprint 

```quantum_correction``` function

In [ ]:
n_q = len(diagonalized_generators_GuA[0])

rotated_H = deepcopy(ham) ##<-- full Hamiltonian

# iteratively perform R rotation over all terms in orginal Hamiltonian
for R in Rotations_list:
    newly_rotated_H={}
    for P in rotated_H.keys():
        lin_comb_Rot_P = c.apply_rotation(R,P) # linear combination of Paulis from R rotation on P
        
        for P_rot in lin_comb_Rot_P:
            
            if P_rot in newly_rotated_H.keys():
                newly_rotated_H[P_rot]+=lin_comb_Rot_P[P_rot]*rotated_H[P] # already in it hence +=
            else:
                newly_rotated_H[P_rot]=lin_comb_Rot_P[P_rot]*rotated_H[P]
                
    rotated_H = deepcopy(newly_rotated_H) ##<-- perform next R rotation on this H
    
rotated_H     

next find where Z indices in $G'$

In [ ]:
z_indices = []
for d in diagonalized_generators_GuA:
    for i in range(n_q):
        if d[i] == 'Z':
            z_indices.append(i)
            
print(diagonalized_generators_GuA)
print(z_indices)

**The exepcation value of $P_{1}$ terms are $p_{1}= \pm 1$ DETERMINED BY THE NONCONTEXTUAL GROUND STATE**

In [ ]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

We need to ENFORCE the diagnal geneators assigned values in the diagonal basis to these expectation values above^^^

In [ ]:
ham_red = {}
    
for P in rotated_H.keys():
    sgn = 1
    for j, z_index in enumerate(z_indices): # enforce diagonal generator's assigned values in diagonal basis
        if P[z_index] == 'Z':
            sgn = sgn*eigen_vals_nonC_ground_state_GuA_ops[j] #<- eigenvalue of nonC ground state!
        elif P[z_index] != 'I':
            sgn = 0

    if sgn != 0:
        # construct term in reduced Hilbert space
        P_red = ''
        for i in range(n_q):
            if not i in z_indices:
                P_red = P_red + P[i]
        if P_red in ham_red.keys():
            ham_red[P_red] = ham_red[P_red] + rotated_H[P]*sgn
        else:
            ham_red[P_red] = rotated_H[P]*sgn
            
ham_red

In [ ]:
c.quantum_correction(ham, #<- full Ham
                     model,
                     fn_form,
                     ground_state_params)

In [ ]:
c.quantum_correction(nonCon_H,model,fn_form,ground_state_params)

In [ ]:
c.get_reduced_hamiltonians(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           list(range(hamiltonians[mol_key][1])))[-1] == rotated_H ### aka when considering all qubit problem it is equal to rotated H!

For some reason it seems that when considering full Hamiltonian there is no reduction in the number of terms!

Q. Do you expect any term reduction when doing CS-VQE?

In [ ]:
n2 = hamiltonians[mol_key][1]-len(diagonalized_generators_GuA)
n2

In [ ]:
ham_red

In [ ]:
ham==Con_H

In [ ]:
n_q = len(diagonalized_generators_GuA[0])
rotated_Hcon = deepcopy(Con_H)

# iteratively perform R rotation over all terms in orginal Hamiltonian
for R in Rotations_list:
    newly_rotated_H={}
    for P in rotated_Hcon.keys():
        lin_comb_Rot_P = c.apply_rotation(R,P) # linear combination of Paulis from R rotation on P
        
        for P_rot in lin_comb_Rot_P:
            
            if P_rot in newly_rotated_H.keys():
                newly_rotated_H[P_rot]+=lin_comb_Rot_P[P_rot]*rotated_Hcon[P] # already in it hence +=
            else:
                newly_rotated_H[P_rot]=lin_comb_Rot_P[P_rot]*rotated_Hcon[P]
                
    rotated_Hcon = deepcopy(newly_rotated_H) ##<-- perform next R rotation on this H
    
rotated_Hcon

In [ ]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

p1_dict = {Gener.index('Z'): p1 for Gener, p1 in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops)}
p1_dict

In [ ]:
new={}
for P1_P2 in rotated_Hcon.keys():
    Z_indices = [i for i, sigma in enumerate(P1_P2) if sigma=='Z']
    
    I1_P2=list(deepcopy(P1_P2))
    sign=1
    for ind in Z_indices:
        sign*=p1_dict[ind]
        I1_P2[ind]='I'
    I1_P2=''.join(I1_P2)
    
    new[I1_P2]=rotated_Hcon[P1_P2]*sign
    
new    

In [ ]:
len(rotated_Hcon)-len(diagonalized_generators_GuA)
# len(new)

In [ ]:
H = conv_scr.Get_Operfermion_Hamiltonian(new)
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

In [ ]:
H_reduced_subspace={}
for P in rotated_H.keys():
    
    sign=1
    for P_known, eigen_val in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops):
        Z_index = P_known.index('Z') # Find single qubit Z in generator!
        
        if P[Z_index]== 'Z': # compare location in genertor to P of rotated H
            
            sign*=eigen_val #<- eigenvalue of nonC ground state!
            
        elif P[Z_index]!= 'I': 
            sign=0 # MUST anti-commute!
    
    # build reduced Hilbert Space
    if sign!=0:
        P_new = list(deepcopy(P))
        P_new[Z_index]='I'
        P_new= ''.join(P_new)
        
        if P_new in H_reduced_subspace.keys():
            H_reduced_subspace[P_new] = H_reduced_subspace[P_new] + rotated_H[P]*sign
        else:
            H_reduced_subspace[P_new] = rotated_H[P]*sign
            
#     else:
#         H_reduced_subspace[P]=rotated_H[P]

In [ ]:
print(len(rotated_H))
print(len(H_reduced_subspace))

In [ ]:
# H_reduced_subspace

In [ ]:
lowest_eigenvalue

In [ ]:
from openfermion import qubit_operator_sparse
import conversion_scripts as conv_scr
import scipy as sp
H = conv_scr.Get_Operfermion_Hamiltonian(H_reduced_subspace)
sparseH = qubit_operator_sparse(H, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

In [ ]:
c.quantum_correction(ham, #<- full Ham
                     model,
                     fn_form,
                     ground_state_params)

In [ ]:
lowest_eigenvalue

In [ ]:
Hfull = conv_scr.Get_Operfermion_Hamiltonian(ham)
sparseHfull = qubit_operator_sparse(Hfull, n_qubits=hamiltonians[mol_key][1])
FCI = sp.sparse.linalg.eigsh(sparseHfull, which='SA', k=1)[0][0]
print('FCI=', FCI)

In [ ]:
sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]

In [27]:
print(diagonalized_generators_GuA)
print(eigen_vals_nonC_ground_state_GuA_ops)

p1_dict = {Gener.index('Z'): p1 for Gener, p1 in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops)}
p1_dict

NameError: name 'diagonalized_generators_GuA' is not defined

In [ ]:
H_reduced_subspace={}
for P in rotated_Hcon.keys():
    
    new_sign=1
    P_new = list(P)
    for index, sigma in enumerate(P):
        if sigma == 'Z':
            new_sign*=p1_dict[index]
            P_new[index]='I'
    
    P_new = ''.join(P_new)
    H_reduced_subspace[P_new] = rotated_Hcon[P]*new_sign

In [ ]:
H_reduced_subspace

H_con_subspace = conv_scr.Get_Operfermion_Hamiltonian(H_reduced_subspace)
sparseH_con_subspace = qubit_operator_sparse(H_con_subspace, n_qubits=hamiltonians[mol_key][1])
sp.sparse.linalg.eigsh(sparseH_con_subspace, which='SA', k=1)[0][0]

In [ ]:
# H_reduced_subspace={}
# for P in rotated_Hcon.keys():
    
#     p1=1
#     for P_known, eigen_val in zip(diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops):
#         Z_index = P_known.index('Z') # Find single qubit Z in generator!
        
#         if P[Z_index]== 'Z': # compare location in genertor to P of rotated H
#             p1*=eigen_val #<- eigenvalue of nonC ground state!
            
#             P1_P2 = list(deepcopy(P))
#             P1_P2[Z_index]='I'
#             I1_P2= ''.join(P1_P2)
            
#             if I1_P2 in H_reduced_subspace.keys():
#                 H_reduced_subspace[I1_P2] += rotated_Hcon[P]*p1
#             else:
#                 H_reduced_subspace[I1_P2] = rotated_Hcon[P]*p1
            
#         elif P[Z_index]== 'I':
#             H_reduced_subspace[P] = rotated_Hcon[P]
            
#         elif P[Z_index]!= 'I': 
#             sign=0 # MUST anti-commute!
#             H_reduced_subspace[P]=0
    
# #         # build reduced Hilbert Space
# #         if sign!=0:
# #             if P_new in H_reduced_subspace.keys():
# #                 H_reduced_subspace[P_new] = H_reduced_subspace[P_new] + rotated_Hcon[P]*sign
# #             else:
# #                 H_reduced_subspace[P_new] = rotated_Hcon[P_new]*sign
            
# # #     else:
# # #         H_reduced_subspace[P]=rotated_H[P]

In [ ]:
H_reduced_subspace

In [ ]:
nonCon_Energy = lowest_eigenvalue

H_con_subspace = conv_scr.Get_Operfermion_Hamiltonian(H_reduced_subspace)
sparseH_con_subspace = qubit_operator_sparse(H_con_subspace, n_qubits=hamiltonians[mol_key][1])
Con_Energy = sp.sparse.linalg.eigsh(sparseH_con_subspace, which='SA', k=1)[0][0]

Con_Energy+nonCon_Energy

In [ ]:
FCI

In [ ]:
c.quantum_correction(ham, #<- full Ham
                     model,
                     fn_form,
                     ground_state_params)

In [ ]:
FCI-lowest_eigenvalue

In [ ]:
Con_Energy

In [ ]:
c.commute(P_gen, P)